In [ ]:
import psycopg2
import random
from datetime import datetime
import time
import os
from faker import Faker
import pandas as pd
from dotenv import load_dotenv

def run_rules(df):
    df['rules_triggered'] = None
    df['rules_explanation'] = None
    df['decision'] = None
    
    if df['amount'][0] >= 100 and df['account_blacklisted'][0] == False and df['trans_type'][0]=='Real_time_transaction':
        df['rules_triggered'] = 'Rule 1'
        df['rules_explanation'] = 'User is making a transaction of more than 100'
        df['decision'] = 'Rejected'
        
    elif df['account_blacklisted'][0] == True and df['trans_type'][0]=='Real_time_transaction':
        df['rules_triggered'] = 'Rule 2'
        df['rules_explanation'] = 'Card is blacklisted'
        df['decision'] = 'Rejected'
    
    elif df['trans_type'][0]=='Real_time_transaction':
        df['rules_explanation'] = 'No rules triggered'
        df['decision'] = 'Approved'
    
    else:
        df['rules_triggered'] = 'No rules triggered'
    
    dict_index = df.to_dict(orient='index')
    dict_single_row = dict_index[list(dict_index.keys())[0]]
    return dict_single_row

In [ ]:
#creating fake dataset
fake = Faker()

num_records = 10 #genreating 10 records
merchant_categories = [
    "Retail","Electronics","Clothing","Groceries","Pharmacy","Entertainment","Healthcare","Dining","Travel","Utilities"
]
card_type = {
    "visa":"visa",
    "mastercard": "Mastercard",
}

host = "ADD YOUR HOST ENDPOINT" #make sure it is public and has inbound port 5432 open for postgres  
port=5432
dbname="postgres"
user="postgres"
password="postgres"

conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)
cur = conn.cursor()

#create banking daa table if it does not exist
create_table_query = """
CREATE TABLE IF NOT EXISTS banking_data (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP WITH TIME ZONE NOT NULL,
    uniq_id UUID NOT NULL,
    trans_type VARCHAR(50) NOT NULL,
    amount DECIMAL(10,2) NOT NULL,
    amount_crr DECIMAL(10,2) NOT NULL,
    account_holder_name VARCHAR(100) NOT NULL,
    card_presence VARCHAR(50) NOT NULL,
    merchant_category VARCHAR(50) NOT NULL,
    card_type VARCHAR(50) NOT NULL,
    card_id VARCHAR(50) NOT NULL,
    account_id UUID NOT NULL,
    account_blacklisted BOOLEAN NOT NULL,
    rules_triggered VARCHAR(100),
    rules_explanation VARCHAR(100),
    decision VARCHAR(100) )
"""

cur.execute(create_table_query)
conn.commit()

#generate fake data
def generate_record():
    cardtype = random.choice(list(card_type.keys()))
    return {
        "uniq_id": [fake.uuid4()],
        "trans_type":[random.choice(["Real_time_transaction","settlements","dispute"])],
        "amount":[round(random.uniform(10.0,1000.0),2)],
        "amount_crr":[round(random.uniform(10.0,1000.0),2)],
        "account_holder_name":[fake.name()],
        "card_presence":[random.choice(["present","absent"])],
        "merchant_category":[random.choice(merchant_categories)],
        "card_type":[cardtype],
        "card_id":[fake.credit_card_number(cardtype)],
        "account_id":[fake.uuid4()],
        "account_blacklisted":[random.choice([True,False])],
    }
    
while True:
    timestamp = datetime.now()
    
    for _ in range(num_records):
        temp = generate_record()
        df = pd.DataFrame(temp)
        record = run_rules(df)
        
        cur.execute(
            """
            INSERT INTO banking_data (
                timestamp, uniq_id,
                trans_type, amount, amount_crr, account_holder_name,
                card_presence, merchant_category, card_type, card_id,
                account_id, account_blacklisted, rules_triggered, rules_explanation, decision)   
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                timestamp, record["uniq_id"],
                record["trans_type"], record["amount"], record["amount_crr"], record["account_holder_name"],
                record["card_presence"], record["merchant_category"], record["card_type"], record["card_id"],
                record["account_id"], record["account_blacklisted"], record["rules_triggered"], record["rules_explanation"], record["decision"]
            )
        )
        conn.commit()
        time.sleep(15)     
cur.close()
conn.close()


KeyboardInterrupt: 